<a href="https://colab.research.google.com/github/Requenamar3/Data-Mining/blob/main/Module4.%20Online%20Retail%20HW%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running the Apriori Algorithm and Finding Association Rules in Online Retail Datasets


**Apply the Apriori algorithm and association rule mining to areal-world dataset**

##Data Loading

In [ ]:
!pip install mlxtend -qqq

The command !pip install mlxtend -qqq silently installs the mlxtend library in the Jupyter notebook. The mlxtend package provides additional tools for data science and machine learning tasks. The -qqq option makes the installation process quiet, minimizing output to keep the environment tidy.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mlxtend.frequent_patterns
import mlxtend.preprocessing
import numpy
import datetime as dt
!pip install matplotlib
retail = pd.read_excel(
    io="https://github.com/fenago/datasets/raw/main/Online%20Retail%20(5).xlsx",
    sheet_name="Online Retail",
    header=0
)

# EDA

In [ ]:
 # installing the pandas_profiling package for data analysis and generating statistical report summaries.
!pip install ydata_profiling

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.9/357.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 20.0 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=40586e2fa342abaeceefc98f4dde05510c36e132f6bb98916883a6b3751c1e78
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.13.1
    Uninstalling seaborn-0.13.1:
      Successfully uninstalled seaborn-0.13.1


In [ ]:
retail.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#number of rows and columns
print("Data dimension (row count, col count): {dim}".format(dim=retail.shape))

# prints the number of unique invoice numbers present in the InvoiceNo
print("Count of unique invoice numbers: {cnt}".format(cnt=retail.InvoiceNo.nunique()))

#prints the count of unique customer IDs in the CustomerID
print("Count of unique customer ids: {cnt}".format(cnt=retail.CustomerID.nunique()))

Data dimension (row count, col count): (541909, 8)
Count of unique invoice numbers: 25900
Count of unique customer ids: 4372


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
unique_invoice_nos = retail['InvoiceNo'].unique()

# Displaying the first 50 unique InvoiceNos
first_50_unique_invoice_nos = unique_invoice_nos[:50]
print(first_50_unique_invoice_nos)

[536365 536366 536367 536368 536369 536370 536371 536372 536373 536374
 536375 536376 536377 536378 536380 536381 'C536379' 536382 'C536383'
 536384 536385 536386 536387 536388 536389 536390 'C536391' 536392 536393
 536394 536395 536396 536397 536398 536399 536400 536401 536402 536403
 536404 536405 536406 536407 536408 536409 536412 536414 536415 536416
 536420]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
invoice_numbers_to_display = ['C536379', 'C536383', 'C536391']
records_with_c = retail[retail['InvoiceNo'].isin(invoice_numbers_to_display)]
print(records_with_c)


    InvoiceNo StockCode                        Description  Quantity  \
141   C536379         D                           Discount        -1   
154   C536383    35004C    SET OF 3 COLOURED  FLYING DUCKS        -1   
235   C536391     22556     PLASTERS IN TIN CIRCUS PARADE        -12   
236   C536391     21984   PACK OF 12 PINK PAISLEY TISSUES        -24   
237   C536391     21983   PACK OF 12 BLUE PAISLEY TISSUES        -24   
238   C536391     21980  PACK OF 12 RED RETROSPOT TISSUES        -24   
239   C536391     21484        CHICK GREY HOT WATER BOTTLE       -12   
240   C536391     22557   PLASTERS IN TIN VINTAGE PAISLEY        -12   
241   C536391     22553             PLASTERS IN TIN SKULLS       -24   

            InvoiceDate  UnitPrice  CustomerID         Country  
141 2010-12-01 09:41:00      27.50     14527.0  United Kingdom  
154 2010-12-01 09:49:00       4.65     15311.0  United Kingdom  
235 2010-12-01 10:24:00       1.65     17548.0  United Kingdom  
236 2010-12-01 10:2

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Data Cleaning and Formatting

In [ ]:
# adds new column to the 'retail' dataframe named 'IsCPresent'.

retail['IsCPresent'] = (
    # Start with the 'InvoiceNo' column of the 'retail' dataframe
    retail['InvoiceNo']
    # Convert the 'InvoiceNo' column to string type to enable string operations
    .astype(str)
    # Apply a lambda function to each element in the 'InvoiceNo' column
    .apply(lambda x:
           # If 'C' is found in the invoice number string, set the element to 1
           1 if x.find('C') != -1
           # Otherwise, if 'C' is not found, set the element to 0
           else 0
          )
)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
retail.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,IsCPresent
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0


In [ ]:
# new data frame retail1 to hold the cleaned or transformed data from the original df retail
retail1 = (
    retail
    # Filter out non-positive quantity values.
    .loc[retail["Quantity"] > 0]
    # Remove InvoiceNos starting with 'C' by checking a flag 'IsCPresent' that indicates such values.
    .loc[retail['IsCPresent'] != 1]
    # Keep only the 'InvoiceNo' and 'Description' columns.
    .loc[:, ["InvoiceNo", "Description"]]
    # Drop all rows with at least one missing value in any of the remaining columns.
    .dropna()
)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
retail1.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,Description
0,536365,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,WHITE METAL LANTERN
2,536365,CREAM CUPID HEARTS COAT HANGER
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,RED WOOLLY HOTTIE WHITE HEART.
5,536365,SET 7 BABUSHKA NESTING BOXES
6,536365,GLASS STAR FROSTED T-LIGHT HOLDER
7,536366,HAND WARMER UNION JACK
8,536366,HAND WARMER RED POLKA DOT
9,536367,ASSORTED COLOUR BIRD ORNAMENT


In [ ]:
# count of rows and columns in the dataframe.
print("Data dimension (row count, col count): {dim}".format(dim=retail1.shape))

# number of unique transactions present in the data.
print("Count of unique invoice numbers: {cnt}".format(cnt=retail1.InvoiceNo.nunique()))


Data dimension (row count, col count): (530693, 2)
Count of unique invoice numbers: 20136


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# extract unique invoice numbers as list
# convert the 'InvoiceNo' column to a list
invoice_no_list = retail1.InvoiceNo.tolist()
# use 'set' to remove any duplicates, converting the list back to a list type
invoice_no_list = list(set(invoice_no_list))

# print the length of the list/ the count of unique invoice numbers
print("Length of list of invoice numbers: {ln}".format(ln=len(invoice_no_list)))


Length of list of invoice numbers: 20136


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


By converting the column to a set, it ensures that all duplicates are removed since a set can only contain unique elements. Converting the set back to a list allows for potential further operations that require list functionality.

In [ ]:
# Selecting the first 5000 unique invoice numbers from the list to create a subset
subset_invoice_no_list = invoice_no_list[0:5000]

# Print the length of the subset list to confirm the number of invoice numbers it contains
print("Length of subset list of invoice numbers: {ln}".format(ln=len(subset_invoice_no_list)))


Length of subset list of invoice numbers: 5000


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


This smaller subset is useful for exercises, demonstrations, or performance reasons when working with large datasets.

In [ ]:
# filter data set down to based on
# subset of invoice number list
retail1 = retail1.loc[retail1["InvoiceNo"].isin(subset_invoice_no_list)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
retail1.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,Description
0,536365,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,WHITE METAL LANTERN
2,536365,CREAM CUPID HEARTS COAT HANGER
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,RED WOOLLY HOTTIE WHITE HEART.
5,536365,SET 7 BABUSHKA NESTING BOXES
6,536365,GLASS STAR FROSTED T-LIGHT HOLDER
7,536366,HAND WARMER UNION JACK
8,536366,HAND WARMER RED POLKA DOT
9,536367,ASSORTED COLOUR BIRD ORNAMENT


In [ ]:
retail1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133315 entries, 0 to 136244
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   InvoiceNo    133315 non-null  object
 1   Description  133315 non-null  object
dtypes: object(2)
memory usage: 3.1+ MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print("Data dimension (row count, col count): {dim}".format(dim=retail1.shape))
print("Count of unique invoice numbers: {cnt}".format(cnt=retail1.InvoiceNo.nunique()))

Data dimension (row count, col count): (133315, 2)
Count of unique invoice numbers: 5000


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
invoice_item_list = []
for num in list(set(retail1.InvoiceNo.tolist())):
    # filter data set down to one invoice number
    tmp_df = retail1.loc[retail1['InvoiceNo'] == num]
    # extract item descriptions and convert to list
    tmp_items = tmp_df.Description.tolist()
    # append list invoice_item_list
    invoice_item_list.append(tmp_items)

print(invoice_item_list[1:5])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#change data type to string
retail['InvoiceNo'] = retail['InvoiceNo'].astype(str)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
retail.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
#change data type and merge invoiceno and stockcode to form a unique identifier

retail['InvoiceNo'] = retail['InvoiceNo'].astype(str)
retail['StockCode'] = retail['StockCode'].astype(str)
retail['transaction_id'] = retail['InvoiceNo'] + '_' + retail['StockCode']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#remove unused columns
retail.drop(['Country', 'Description','CustomerID'], axis=1, inplace=True)

retail.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,transaction_id
0,536365,85123A,6,2010-12-01 08:26:00,2.55,536365_85123A
1,536365,71053,6,2010-12-01 08:26:00,3.39,536365_71053
2,536365,84406B,8,2010-12-01 08:26:00,2.75,536365_84406B
3,536365,84029G,6,2010-12-01 08:26:00,3.39,536365_84029G
4,536365,84029E,6,2010-12-01 08:26:00,3.39,536365_84029E


In [ ]:
retail.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


InvoiceNo         0
StockCode         0
Quantity          0
InvoiceDate       0
UnitPrice         0
transaction_id    0
dtype: int64

##Part 1: Running the Apriori Algorithm on the Complete Online Retail Dataset

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

# One-hot encoding the dataset
basket = (retail
          .groupby(['InvoiceNo', 'StockCode'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

# Convert the units to 1s and 0s
basket_sets = basket.applymap(lambda x: 1 if x > 0 else 0)

# Applying Apriori
# Use the apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

# Generating Rules
# Generate the rules with their corresponding support, confidence, and lift
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# Display the top 10 rules
rules.sort_values('lift', ascending=False).head(10)


### 1.Run the Apriori Algorithm: Apply the Apriori algorithm to the full dataset. Set parameters to identify item sets appearing in at least 1% of transactions.


### 2.Analysis of Specific Item Set: Filter to the item set containing "10 COLOUR SPACEBOY PEN." Analyze its support value.

### 3.Item Set Length Analysis: Add a column for item set length. Focus on sets of length two with support between 0.02 and 0.021. Compare these to previous results.

### 4.Support Value Distribution: Plot the distribution of support values for the identified item sets

##Part 2: Finding the Association Rules on the Complete Online Retail Dataset

### 1.Fit Association Rule Model: Use the full dataset to generate association rules. Apply the confidence metric with a minimum threshold of 0.6.

### 2.Count and Compare Association Rules: Count the number of derived rules. Compare this with previous exercises.

### 3.Plot Confidence vs. Support: Create a plot to visualize the relationship between confidence and support for the rules.

### 4.Analyze Rule Metrics: Examine the distributions of confidence, lift, leverage, and conviction in your rules.

##Report writing



*   Insights gained from the Apriori algorithm and association rule mining.
*   How these insights can influence business decisions like store layout and pricing.

*   A comparison between the two datasets.
*   Reflections on how data partitioning and increasing data volume impact results.



